In [1]:
# this notebook demos hot to freeze weights pf a model during finetuning
# here we freeze weights of attention layers (not a good idea to do IRL lol)

/Users/raeez/.pyenv/versions/jupyter-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import matplotlib.pyplot as plt

from transformers import AutoModelForCausalLM, GPT2Tokenizer
import requests



In [2]:
gpt2  = AutoModelForCausalLM.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

Loading weights: 100%|█| 148/148 [00:00<00:00, 1756.78it/s, Materializing param=
GPT2LMHeadModel LOAD REPORT from: gpt2
Key                  | Status     |  | 
---------------------+------------+--+-
h.{0...11}.attn.bias | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [3]:
seq_len = 256
batch_size = 16

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

In [5]:
# tokenize the text
text = requests.get('https://www.gutenberg.org/files/829/829-0.txt').text
# text = requests.get('https://www.gutenberg.org/cache/epub/829/pg289.txt').text
gtTokens = tokenizer.encode(text, return_tensors='pt')[0] # Now this outputs a Pytorch tensor ('pt')
print(gtTokens.shape)


torch.Size([143728])


Freeze all attention head weeights

In [7]:
for name, param in gpt2.named_parameters():
    print(name)
# this code lists out all the params(weights) f the model

transformer.wte.weight
transformer.wpe.weight
transformer.h.0.ln_1.weight
transformer.h.0.ln_1.bias
transformer.h.0.attn.c_attn.weight
transformer.h.0.attn.c_attn.bias
transformer.h.0.attn.c_proj.weight
transformer.h.0.attn.c_proj.bias
transformer.h.0.ln_2.weight
transformer.h.0.ln_2.bias
transformer.h.0.mlp.c_fc.weight
transformer.h.0.mlp.c_fc.bias
transformer.h.0.mlp.c_proj.weight
transformer.h.0.mlp.c_proj.bias
transformer.h.1.ln_1.weight
transformer.h.1.ln_1.bias
transformer.h.1.attn.c_attn.weight
transformer.h.1.attn.c_attn.bias
transformer.h.1.attn.c_proj.weight
transformer.h.1.attn.c_proj.bias
transformer.h.1.ln_2.weight
transformer.h.1.ln_2.bias
transformer.h.1.mlp.c_fc.weight
transformer.h.1.mlp.c_fc.bias
transformer.h.1.mlp.c_proj.weight
transformer.h.1.mlp.c_proj.bias
transformer.h.2.ln_1.weight
transformer.h.2.ln_1.bias
transformer.h.2.attn.c_attn.weight
transformer.h.2.attn.c_attn.bias
transformer.h.2.attn.c_proj.weight
transformer.h.2.attn.c_proj.bias
transformer.h.2.ln_2

In [8]:
for name, param in gpt2.named_parameters():
    if '.h.' in name:
        param.requires_grad = False
        print(f'---- Layer {name} is frozen (.requires_grad = {param.requires_grad}).')
    elif '.h.' not in name:
        print(f'---- Layer {name} is trainabel (.requires_grad = {param.requires_grad}).')

---- Layer transformer.wte.weight is trainabel (.requires_grad = True).
---- Layer transformer.wpe.weight is trainabel (.requires_grad = True).
---- Layer transformer.h.0.ln_1.weight is frozen (.requires_grad = False).
---- Layer transformer.h.0.ln_1.bias is frozen (.requires_grad = False).
---- Layer transformer.h.0.attn.c_attn.weight is frozen (.requires_grad = False).
---- Layer transformer.h.0.attn.c_attn.bias is frozen (.requires_grad = False).
---- Layer transformer.h.0.attn.c_proj.weight is frozen (.requires_grad = False).
---- Layer transformer.h.0.attn.c_proj.bias is frozen (.requires_grad = False).
---- Layer transformer.h.0.ln_2.weight is frozen (.requires_grad = False).
---- Layer transformer.h.0.ln_2.bias is frozen (.requires_grad = False).
---- Layer transformer.h.0.mlp.c_fc.weight is frozen (.requires_grad = False).
---- Layer transformer.h.0.mlp.c_fc.bias is frozen (.requires_grad = False).
---- Layer transformer.h.0.mlp.c_proj.weight is frozen (.requires_grad = False).

In [9]:
# as a sanity check grab wieghts from oen frozen and one trainable layer
frozenW_pre = gpt2.transformer.h[6].mlp.c_fc.weight.data
trainW_pre = gpt2.transformer.ln_f.weight.data

In [11]:
gpt2 = gpt2.to(device)

FIne tune the model

In [12]:
optimizer = torch.optim.AdamW(gpt2.parameters(), lr=5e-5, weight_decay=.01)



In [13]:
num_samples = 100

#init the loss
train_loss = []


for sampli in range(num_samples):
    ix = torch.randint(len(gtTokens)-seq_len,size=(batch_size,))
    X = gtTokens[ix[:,None]+ torch.arange(seq_len)].to(device)
    
    gpt2.zero_grad()
    # fwd pass (HF shifts X internally to get y)
    # all of X is shifted by 1 and used as labels for loss calcualtion
    # also inside model, H.F makes sure it uses NLLLoss as loss func
    # fwd pass
    outputs= gpt2(X,labels=X)
    loss = outputs.loss
    loss.backward()
    optimizer.step()
    
    # sum the batch loss
    train_loss.append(loss.item())
    if sampli%77==0:
        print(f'Sample {sampli}/{num_samples}, train loss: {train_loss[-1]}')

`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Sample 0/100, train loss: 4.0601487159729
Sample 77/100, train loss: 3.632295608520508


In [24]:
# as a sanity check grab wieghts from oen frozen and one trainable layer
frozenW_pst = gpt2.transformer.h[6].mlp.c_fc.weight.data.cpu()
trainW_pst = gpt2.transformer.ln_f.weight.data.cpu()

In [25]:
print(frozenW_pre.shape)
print(frozenW_pst.shape)

torch.Size([768, 3072])
torch.Size([768, 3072])


In [26]:
# to chek if 2 matrix are equal we take their norm of difference
# it should be zeros if the layer was frozen
print('Frozen layer, norm(post-pre):')
print('  ',torch.norm(frozenW_pst-frozenW_pre))

print('\n\nTrainable  layer, norm(post-pre):')
print('  ',torch.norm(trainW_pst-trainW_pre))


Frozen layer, norm(post-pre):
   tensor(0.)


Trainable  layer, norm(post-pre):
   tensor(0.0818)
